# Get Atomref Energies for Psi4
SchNet normalizes the output energies using the energies of isolated atoms, which it terms "atomrefs". This notebook computes those for Psi4 and then updates the network accordingly

In [1]:
from schnetpack.interfaces.ase_interface import SpkCalculator
from ase.calculators.psi4 import Psi4
from ase.db import connect
from ase import Atoms, units, build
import numpy as np
import torch

/home/lward/miniconda3/envs/fastforce/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get the isolated atom energies
Used to normalize the energies of SchNetPack models

In [2]:
atomref = {}
for elem, multi in {'H': 2, 'O': 3}.items():
    calc = Psi4(method='b3lyp-d3', basis='6-31g', multiplicity=multi, reference='uhf')
    atoms = Atoms(symbols=elem, positions=[[0,0,0]])
    eng = calc.get_potential_energy(atoms)
    atomref[elem] = eng
atomref

  Threads set to 1 by Python driver.
  Threads set to 1 by Python driver.


/home/lward/miniconda3/envs/fastforce/lib/python3.10/site-packages/psi4/driver/procrouting/empirical_dispersion.py:212: FutureWarning: Using the `local_options` keyword argument is deprecated in favor of using `task_config`, in version 0.30.0 it will stop working.
  jobrec = qcng.compute(
/home/lward/miniconda3/envs/fastforce/lib/python3.10/site-packages/psi4/driver/procrouting/empirical_dispersion.py:212: FutureWarning: Using the `local_options` keyword argument is deprecated in favor of using `task_config`, in version 0.30.0 it will stop working.
  jobrec = qcng.compute(


{'H': -13.61313070407502, 'O': -2042.4411812386718}

## Update these values in the network
The atomrefs are stored as an "Embedding" layer

In [3]:
model = torch.load('../train-schnetpack/test/best_model', map_location='cpu')

Make a calculator to show how poorly it works

In [4]:
water = build.molecule('H2O')

In [5]:
psi4_eng = Psi4(method='b3lyp-d3', basis='6-31g').get_potential_energy(water)

/home/lward/miniconda3/envs/fastforce/lib/python3.10/site-packages/psi4/driver/procrouting/empirical_dispersion.py:212: FutureWarning: Using the `local_options` keyword argument is deprecated in favor of using `task_config`, in version 0.30.0 it will stop working.
  jobrec = qcng.compute(


In [6]:
spk_calc = SpkCalculator(model, energy='energy', forces='forces')
spk_eng = spk_calc.get_potential_energy(atoms)

In [7]:
print(f'Energy of water - Psi4: {psi4_eng:.2f} eV, SchNet {spk_eng:.2f} eV')

Energy of water - Psi4: -2078.55 eV, SchNet -0.11 eV


Huge difference. Let's adjust SchNet so that it adds in isolated energies for O and H to the energy

We do that by a vector of reference energies where the key is the atomic number, then adding it to the network

In [8]:
atomrefs = np.zeros((32,), dtype=np.float32)
atomrefs[1] = atomref['H']
atomrefs[8] = atomref['O']

In [9]:
before_atomrefs = model.output_modules[0].atomref
model.output_modules[0].atomref = torch.nn.Embedding.from_pretrained(torch.from_numpy(atomrefs[:, None]))

In [10]:
spk_calc = SpkCalculator(model, energy='energy', forces='forces')
spk_eng_new = spk_calc.get_potential_energy(atoms)

In [11]:
print(f'Energy of water - Psi4: {psi4_eng:.2f} eV, SchNet {spk_eng_new:.2f} eV')

Energy of water - Psi4: -2078.55 eV, SchNet -2042.55 eV


They are now much closer. There are differences, presumably due to bond length issues, but we are at least in the right ballpark

## Update the mean and standard deviation
These are other features that will adjust for TTM and DFT having different energies per water (0 for TTM, ~-9 eV for others)

In [12]:
dft_energy_per_atom = []
with connect('../initial-database/initial-psi4-631g.db') as db:
    dft_energy_pa = [(a.energy - sum(map(atomref.get, a.symbols))) / a.natoms  for a in db.select('')]

The mean and standard deviation are in the standardization layer

In [13]:
std_layer = before_atomrefs = model.output_modules[0].standardize
print(f'Values with TTM - Mean {std_layer.mean[0]:.2f}, St. Dev. {std_layer.stddev[0]:.3f}')

Values with TTM - Mean -0.13, St. Dev. 0.005


In [14]:
std_layer.mean = torch.from_numpy(np.mean(dft_energy_pa)[None])
std_layer.stddev = torch.from_numpy(np.std(dft_energy_pa)[None])
print(f'Values with DFT - Mean {std_layer.mean[0]:.2f}, St. Dev. {std_layer.stddev[0]:.3f}')

Values with DFT - Mean -3.16, St. Dev. 0.084


We will now have a ~3 eV offset per atom, which accounts for the different in bonding

In [15]:
spk_calc = SpkCalculator(model, energy='energy', forces='forces')
spk_eng_new = spk_calc.get_potential_energy(atoms)

In [16]:
print(f'Energy of water - Psi4: {psi4_eng:.2f} eV, SchNet {spk_eng_new:.2f} eV')

Energy of water - Psi4: -2078.55 eV, SchNet -2045.26 eV


## Save Updated Model
For us to use later

In [17]:
torch.save(model, 'starting-psi4-model')